In [2]:
import mlflow
import pandas as pd
import numpy as np


In [3]:
vald_data = pd.read_csv('data/vald_data_for_modelling.csv')

In [4]:
import mlflow
import mlflow.sklearn
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd

# Load dataset
vald_data = pd.read_csv("data/vald_data_for_modelling.csv")

# Define features and target
risk_mapping = {'Low Risk': 0, 'Medium Risk': 1, 'High Risk': 2}
X = vald_data[['ForceSymmetry', 'MaxForceSymmetry', 'TorqueSymmetry']]
y = vald_data['RiskCategory'].map(risk_mapping)

# Set MLflow experiment
mlflow.set_experiment("Athlete_Injury_Risk")

# Function to train and log model with MLflow
def train_and_log_model(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run(run_name=model_name):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        # Log parameters, metrics, and model
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("n_estimators", 100)
        mlflow.log_param("random_state", 42)
        mlflow.log_metric("accuracy", accuracy)
        
        # Log classification report as an artifact
        report = classification_report(y_test, y_pred, target_names=risk_mapping.keys(), output_dict=True)
        mlflow.log_metric("precision_low", report["Low Risk"]["precision"])
        mlflow.log_metric("recall_low", report["Low Risk"]["recall"])
        mlflow.log_metric("f1_low", report["Low Risk"]["f1-score"])
        
        mlflow.log_metric("precision_medium", report["Medium Risk"]["precision"])
        mlflow.log_metric("recall_medium", report["Medium Risk"]["recall"])
        mlflow.log_metric("f1_medium", report["Medium Risk"]["f1-score"])
        
        mlflow.log_metric("precision_high", report["High Risk"]["precision"])
        mlflow.log_metric("recall_high", report["High Risk"]["recall"])
        mlflow.log_metric("f1_high", report["High Risk"]["f1-score"])

        mlflow.sklearn.log_model(model, model_name)
        
        print(f"{model_name} logged with Accuracy: {accuracy:.4f}")

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Model 1: SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42)
rf_model_1 = RandomForestClassifier(n_estimators=100, random_state=42)
train_and_log_model("RF_SMOTE", rf_model_1, X_train_s, X_test_s, y_train_s, y_test_s)

# Model 2: No Balancing
rf_model_2 = RandomForestClassifier(n_estimators=100, random_state=42)
train_and_log_model("RF_No_Balancing", rf_model_2, X_train, X_test, y_train, y_test)

# Model 3: SMOTEENN
smoteenn = SMOTEENN(random_state=42)
X_resampled_se, y_resampled_se = smoteenn.fit_resample(X, y)
X_train_se, X_test_se, y_train_se, y_test_se = train_test_split(X_resampled_se, y_resampled_se, test_size=0.2, stratify=y_resampled_se, random_state=42)
rf_model_3 = RandomForestClassifier(n_estimators=100, random_state=42)
train_and_log_model("RF_SMOTEENN", rf_model_3, X_train_se, X_test_se, y_train_se, y_test_se)

# Model 4: Class Weights
rf_model_4 = RandomForestClassifier(n_estimators=100, random_state=42, class_weight={0: 1, 1: 2, 2: 3})
train_and_log_model("RF_Class_Weights", rf_model_4, X_train, X_test, y_train, y_test)


2025/01/30 10:19:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RF_SMOTE logged with Accuracy: 0.9839


2025/01/30 10:19:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RF_No_Balancing logged with Accuracy: 0.9897


2025/01/30 10:19:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RF_SMOTEENN logged with Accuracy: 1.0000


2025/01/30 10:19:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RF_Class_Weights logged with Accuracy: 0.9795


In [5]:
import os

from dotenv import load_dotenv
load_dotenv()

True

In [6]:
model_uri=os.getenv("MODEL_URI")
# Load the trained model
best_model = mlflow.pyfunc.load_model(model_uri)
unseen_data=pd.read_csv("data/unseen_data.csv")


In [7]:
X_unseen = unseen_data[['ForceSymmetry', 'MaxForceSymmetry', 'TorqueSymmetry']]
# Make predictions
predictions = best_model.predict(X_unseen)
# Mapping predictions back to Risk Categories
risk_mapping_inverse = {0: 'Low Risk', 1: 'Medium Risk', 2: 'High Risk'}

unseen_data['Predicted Risk']=pd.Series(predictions).map(risk_mapping_inverse)